In [0]:
from delta.tables import *

stage_delta_table = "event_driven_workflow.default.stage_zone_new"
target_delta_table = "event_driven_workflow.default.target_zone_new"

# Read data from stage delta table
stage_df = spark.read.table(stage_delta_table)

# Create target delta table if it doesn't exist
if not spark._jsparkSession.catalog().tableExists(target_delta_table):
    stage_df.write.format("delta").saveAsTable(target_delta_table)
else:
    # Create target delta object
    target_table = DeltaTable.forName(spark, target_delta_table)
    
    # Merge tables based on tracking_num column
    merge_condition = "stage.tracking_num = target.tracking_num"
    
    # Execute the merge operation
    target_table.alias("target") \
        .merge(stage_df.alias("stage"), merge_condition) \
        .whenMatchedDelete() \
        .execute()

    stage_df.write.format("delta").mode("append").saveAsTable(target_delta_table)

